In [1]:
import os
import pandas as pd
import dask.dataframe as dd
import glob
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
dir_path = "/work/group/oit_research_data/mobility/data/data_DFW_2021/data/DFW_2021_01/"
file_name = "G14_DFW"
ext = ".tsv"
file_path = dir_path+file_name+ext

In [3]:
# Function to convert Unix timestamps to seconds (if not already in seconds)
def convert_timestamp_to_dt(ddf, tz='US/Central'):        
    ddf['DateTime'] = dd.to_datetime(ddf['Unix Timestamp'],unit='s')    
    #ddf['DateTime'] = ddf["DateTime"].round("H")
    ddf.DateTime = ddf.DateTime.dt.tz_localize('UTC').dt.tz_convert(tz)  
    #ddf['DateTime']=ddf['DateTime'].astype(str)
    #ddf.DateTime = ddf.DateTime.str.slice(stop=-6)
    ddf['Date'] = ddf.DateTime.dt.date
    return ddf

In [4]:
# Read the large TSV file in chunks using Dask
columns=["Polygon_ID","Device_ID","Lat","Lon","Unix Timestamp"]
df = dd.read_csv(file_path, sep='\t', names=columns, header=None,
                 dtype={"Polygon_ID":'object',"Device_ID":'object',"Lat":'float64',"Lon":'float64',"Unix Timestamp":'object'}).compute()

In [5]:
df

,Polygon_ID,Device_ID,Lat,Lon,Unix Timestamp
0,R1,d2f5d964b8d12b666f884d7b42221a25ceff6368,32.758560,-96.682709,1609538674
1,R1,9802ca185b2baca4c1507a9478d9b6e8c6c118bd,32.760974,-96.682835,1609528435
2,R1,9802ca185b2baca4c1507a9478d9b6e8c6c118bd,32.760656,-96.682824,1609528437
3,R1,9802ca185b2baca4c1507a9478d9b6e8c6c118bd,32.760334,-96.682831,1609528439
4,R1,9802ca185b2baca4c1507a9478d9b6e8c6c118bd,32.759992,-96.682836,1609528441
...,...,...,...,...,...
836564,B17,103565f39bbbf641f634a6953556d7e4df80f0e8,32.853148,-96.586332,1610322448
836565,B17,103565f39bbbf641f634a6953556d7e4df80f0e8,32.853148,-96.586332,1610322494
836566,B17,103565f39bbbf641f634a6953556d7e4df80f0e8,32.853188,-96.586322,1610322539
836567,B17,103565f39bbbf641f634a6953556d7e4df80f0e8,32.853348,-96.586433,1610322584


In [6]:
ddf = dd.from_pandas(df, npartitions=256)

In [7]:
meta_dict = {"Polygon_ID":'object',"Device_ID":'object',"Lat":'float64',"Lon":'float64',"Unix Timestamp":'object', 'DateTime':'object','Date':'object'}

In [8]:
# Convert Unix timestamps to seconds
ddf = ddf.map_partitions(convert_timestamp_to_dt, meta=meta_dict)

In [9]:
ids = pd.read_csv(dir_path+"../finalListOfDeviceIDsAtThreeMonthThreshold.txt")

In [10]:
#np.select(df['Device_ID'].isin(ids),df['Device_ID'])

In [11]:
mob = ddf[ddf.Device_ID.isin(ids.Device_ID)].compute()

In [13]:
mob.to_csv(file_name+'_output.tsv', sep='\t', index=False)